<a href="https://colab.research.google.com/github/HyeJin816/ESAA_22/blob/main/0411.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0411 핸즈온 ch9 (p299~345)

## **9.1 군집**

**군집 Clustering** : 비슷한 샘플을 클러스터 또는 비슷한 샘플의 그룹으로 할당. (비지도 학습)  
> *레이블이 되어 있음* = 로지스틱 회귀, SVM, 랜덤 포레스트 분류기 등의 분류 알고리즘 적합  
> *레이블이 없음* = 군집 알고리즘이 적합  

+ 고객 분류 : 고객 대상 추천 시스템  
+ 데이터 분석 : 군집 알고리즘 실행 후, 각 클러스터 분석
+ 차원 축소 기법 : 각 클러스터에 대한 친화성 측정
+ 이상치 탐지 : 클러스터 친하성으로 결함/부정거래 감지
+ 준지도 학습 : 동일 클러스터 내 레이블 전파
+ 검색 엔진 : 전체 데이터 베이스에 군집 알고리즘 적용
+ 이미지 분할 : 색 기발 픽셀을 클러스터로 모아 물체 탐지 및 추적 시스템 활용  


### **9.1.1 k-평균**

**k-평균** : 몇 번의 반복으로 데이터셋을 빠르고 효율적으로 클러스터로 묶을 수 있음.

In [18]:
import numpy as np
from sklearn.datasets import make_blobs

blob_centers = np.array(
    [[ 0.2,  2.3],
     [-1.5 ,  2.3],
     [-2.8,  1.8],
     [-2.8,  2.8],
     [-2.8,  1.3]])
blob_std = np.array([0.4, 0.3, 0.1, 0.1, 0.1])

X, y = make_blobs(n_samples=2000, centers=blob_centers,
                  cluster_std=blob_std, random_state=7)

In [2]:
from sklearn.cluster import KMeans
k = 5
kmeans = KMeans(n_clusters=k, random_state=42)
y_pred = kmeans.fit_predict(X)
y_pred

array([4, 0, 1, ..., 2, 1, 0], dtype=int32)

In [3]:
y_pred is kmeans.labels_

True

In [4]:
kmeans.cluster_centers_

array([[-2.80389616,  1.80117999],
       [ 0.20876306,  2.25551336],
       [-2.79290307,  2.79641063],
       [-1.46679593,  2.28585348],
       [-2.80037642,  1.30082566]])

In [5]:
# 새로운 샘플에 가까운 센트로이드 클러스터 할당
X_new = np.array([[0, 2], [3, 2], [-3, 3], [-3, 2.5]])
kmeans.predict(X_new)

array([1, 1, 2, 2], dtype=int32)

+ 하드 군집 : 샘플에 가장 가까운 군집을 선택  
+ 소프트 군집 : 클러스터 마다 점수를 부여하여 군집 판단 (유사도 점수/친화성 점수)

In [6]:
# transform() 메서드로 샘플과 센트로이드 사이 거리 반환
kmeans.transform(X_new)

array([[2.81093633, 0.32995317, 2.9042344 , 1.49439034, 2.88633901],
       [5.80730058, 2.80290755, 5.84739223, 4.4759332 , 5.84236351],
       [1.21475352, 3.29399768, 0.29040966, 1.69136631, 1.71086031],
       [0.72581411, 3.21806371, 0.36159148, 1.54808703, 1.21567622]])

#### **k-평균 알고리즘**

1) 센트로이드가 주어져서 데이터셋의 모든 샘플이 센트로이드 클러스에 할당.  
2) 샘플의 레이블이 주어져서 클러스터의 샘플 평균을 통해 센트로이드를 판단.  
> 센트로이드&샘플 모두 주어지지 않으면, 랜덤하게 센트로이드를 선정하여 샘플의 레이블을 할당하고 센트로이드 업데이트를 반복하면서 변화가 줄어들 때까지 반복.

#### **센트로이드 초기화 방법**

1) 센트로이드 위치를 근사하게 알 수 있으면 ```init```에 센트로이드 넘파이 배열을 지정  
2) 랜덤 초기화 횟수를 ```n_init``` 매개변수로 조절하여 여러번 알고리즘을 실행하고 가장 좋은 솔루션 선택.

**이너셔 (inertia)** : 최선의 솔루션 확인 성능 지표. 각 샘플과 가까운 센트로이드 사이의 평균 제곱 거리.  

In [7]:
kmeans.inertia_

211.5985372581684

In [8]:
# 이너셔 음숫값 메서드
kmeans.score(X)

-211.59853725816836

#### **k-평균 속도 개선과 미니배치 k-평균**

삼각 부등식을 사용해 샘플과 센트로이드 사이의 거리를 위한 하한선&상한선 유지를 기본으로.
+ 전체 데이터셋이 아닌 각 반복에서 미니 배치를 사용해서 센트로이드를 조금씩 이동시켜 속도를 3~4배 상승.  
+ 메모리에 들어가지 않는 대량의 데이터셋에 군집 알고리즘 적용 가능  

> 미니배치 k-평균 알고리즘이 훨씬 빠르지만 이너셔는 일반적으로 나쁨 (클러스터 개수가 증가할 경우)

In [9]:
from sklearn.cluster import MiniBatchKMeans

minibatch_kmeans = MiniBatchKMeans(n_clusters=5, random_state=42)
minibatch_kmeans.fit(X)

MiniBatchKMeans(n_clusters=5, random_state=42)

#### **최적의 클러스터 개수 찾기**

k가 증가함에 따라 이너셔가 작아지므로 k선택에 좋은 성능 지표가 아님. >> 다른 방법 필요  
**실루엣 점수** : 모든 샘플에 대한 실루엣 계수의 평균.
> *+1에 근접* = 자신의 클러스터에 잘 속해있으며 다른 클러스터와는 멀리 위치  
> *0에 근접* = 클러스터 경계에 위치  
> *-1에 근접* = 잘못된 클러스터에 할당되어 위치

In [10]:
from sklearn.metrics import silhouette_score

silhouette_score(X, kmeans.labels_)

0.655517642572828

### **9.1.2 k-평균의 한계**

<장점>
> 속도가 빠르고 확장이 용이  

<단점>
> 최적이 아닌 솔루션을 피하기 위해 알고리즘 반복
> 클러스터 개수 지정
> 클러스터 크기나 밀집도가 다른 경우 제대로 작동하지 않음


### **9.1.3 군집을 사용한 이미지 분할**

**이미지 분할 image segmentation** : 이미지를 세그먼트 여러개로 분할하는 작업.  
**시맨틱 분할 semantic segmentation** : 동일한 종류의 물체에 속한 모든 픽셀은 같은 세그먼드에 할당.  
> **색상 분할** : 동일 색상의 픽셀을 같은 세그먼트에 할당

### **9.1.4. 군집을 사용한 전처리**

차원 축소에 효과적인 방법으로 지도 학습 적용 전에 전처리 단계로 사용 가능.

In [29]:
from sklearn.datasets import load_digits
X_digits, y_digits = load_digits(return_X_y=True)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_digits, y_digits)

In [12]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)

log_reg.score(X_test, y_test)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


0.9622222222222222

In [14]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ("kmeans", KMeans(n_clusters=50, random_state=42)),
    ("log_reg", LogisticRegression(random_state=42)),
])
pipeline.fit(X_train, y_train)

pipeline.score(X_test, y_test)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


0.9666666666666667

GridSearchCV를 통해 교차검증에서 가장 좋은 분류 성능을 내는 k값을 확인.

In [20]:
from sklearn.model_selection import GridSearchCV

param_grid = dict(kmeans__n_clusters=range(2, 100))
grid_clf = GridSearchCV(pipeline, param_grid, cv=3, verbose=2)
grid_clf.fit(X_train, y_train)

Fitting 3 folds for each of 98 candidates, totalling 294 fits


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


[CV] END ...............................kmeans__n_clusters=2; total time=   0.3s


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


[CV] END ...............................kmeans__n_clusters=2; total time=   0.4s


...

[CV] END ..............................kmeans__n_clusters=99; total time=   1.1s


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


[CV] END ..............................kmeans__n_clusters=99; total time=   1.3s


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('kmeans',
                                        KMeans(n_clusters=50, random_state=42)),
                                       ('log_reg',
                                        LogisticRegression(random_state=42))]),
             param_grid={'kmeans__n_clusters': range(2, 100)}, verbose=2)

In [21]:
grid_clf.best_params_

{'kmeans__n_clusters': 67}

In [22]:
grid_clf.score(X_test, y_test)

0.9644444444444444

### **9.1.5 군집을 사용한 준지도 학습**

In [23]:
n_labeled = 50
log_reg = LogisticRegression(multi_class="ovr", solver="lbfgs", random_state=42)
log_reg.fit(X_train[:n_labeled], y_train[:n_labeled])

log_reg.score(X_test, y_test)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


0.8

((개선)) **대표 이미지** : 훈련 세트를 50개의 클러스터로 모아서 각 클러스터에서 센트로이드에 가장 가까운 이미지 파악

In [24]:
k = 50
kmeans = KMeans(n_clusters=k, random_state=42)
X_digits_dist = kmeans.fit_transform(X_train)
representative_digit_idx = np.argmin(X_digits_dist, axis=0)
X_representative_digits = X_train[representative_digit_idx]

In [25]:
y_representative_digits = np.array([
    0, 1, 3, 2, 7, 6, 4, 6, 9, 5,
    1, 2, 9, 5, 2, 7, 8, 1, 8, 6,
    3, 1, 5, 4, 5, 4, 0, 3, 2, 6,
    1, 7, 7, 9, 1, 8, 6, 5, 4, 8,
    5, 3, 3, 6, 7, 9, 7, 8, 4, 9])

In [30]:
log_reg = LogisticRegression(multi_class="ovr", solver="lbfgs", max_iter=5000, random_state=42)
log_reg.fit(X_representative_digits, y_representative_digits)

log_reg.score(X_test, y_test)

0.044444444444444446

In [27]:
y_train_propagated = np.empty(len(X_train), dtype=np.int32)
for i in range(k):
    y_train_propagated[kmeans.labels_==i] = y_representative_digits[i]

In [31]:
log_reg = LogisticRegression(multi_class="ovr", solver="lbfgs", max_iter=5000, random_state=42)
log_reg.fit(X_train, y_train_propagated)

log_reg.score(X_test, y_test)

0.09555555555555556

In [32]:
percentile_closest = 20

X_cluster_dist = X_digits_dist[np.arange(len(X_train)), kmeans.labels_]
for i in range(k):
    in_cluster = (kmeans.labels_ == i)
    cluster_dist = X_cluster_dist[in_cluster]
    cutoff_distance = np.percentile(cluster_dist, percentile_closest)
    above_cutoff = (X_cluster_dist > cutoff_distance)
    X_cluster_dist[in_cluster & above_cutoff] = -1
    
partially_propagated = (X_cluster_dist != -1)
X_train_partially_propagated = X_train[partially_propagated]
y_train_partially_propagated = y_train_propagated[partially_propagated]

In [33]:
log_reg = LogisticRegression(multi_class="ovr", solver="lbfgs", max_iter=5000, random_state=42)
log_reg.fit(X_train_partially_propagated, y_train_partially_propagated)

log_reg.score(X_test, y_test)

0.13333333333333333

### **9.1.6 DBSDCAN**

각 샘플에서 작은 거리(입실론) 내에 있는 샘플의 갯수를 세고, min_samples개 이상의 샘플이라면 '핵심 샘플'로 간주하여 그 주변의 샘플을 동일한 클러스터로 할당

In [34]:
from sklearn.datasets import make_moons
X, y = make_moons(n_samples=1000, noise=0.05, random_state=42)

from sklearn.cluster import DBSCAN

dbscan = DBSCAN(eps=0.05, min_samples=5)
dbscan.fit(X)

DBSCAN(eps=0.05)

In [35]:
dbscan.labels_[:10]

array([ 0,  2, -1, -1,  1,  0,  0,  0,  2,  5])

In [36]:
len(dbscan.core_sample_indices_)

808

In [37]:
dbscan.core_sample_indices_ #핵심 샘플 인덱스

array([  0,   4,   5,   6,   7,   8,  10,  11,  12,  13,  14,  16,  17,
        18,  19,  20,  21,  22,  23,  24,  25,  26,  28,  29,  30,  31,
        32,  33,  34,  36,  38,  39,  41,  42,  44,  45,  47,  49,  50,
        51,  52,  53,  54,  55,  56,  58,  59,  61,  63,  64,  65,  66,
        67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,  79,
        80,  81,  83,  84,  85,  87,  88,  89,  90,  91,  93,  94,  96,
        97,  98, 102, 103, 104, 105, 106, 107, 108, 109, 110, 113, 114,
       115, 116, 117, 119, 120, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 135, 136, 139, 140, 141, 143, 144, 145, 146, 147, 148, 149,
       150, 152, 153, 154, 155, 156, 157, 158, 159, 161, 162, 163, 164,
       165, 166, 167, 168, 169, 170, 172, 173, 174, 175, 176, 177, 178,
       179, 181, 182, 183, 185, 186, 187, 188, 189, 191, 193, 194, 195,
       196, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 209, 210,
       211, 212, 213, 214, 215, 216, 217, 218, 219, 221, 222, 22

In [38]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=50)
knn.fit(dbscan.components_, dbscan.labels_[dbscan.core_sample_indices_])

X_new = np.array([[-0.5,0], [0,0.5], [1,-0.1], [2,1]])
knn.predict(X_new)

array([6, 0, 3, 2])

In [39]:
knn.predict_proba(X_new)

array([[0.24, 0.  , 0.  , 0.  , 0.  , 0.  , 0.76],
       [1.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.3 , 0.7 , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 1.  , 0.  , 0.  , 0.  , 0.  ]])

In [40]:
y_dist, y_pred_idx = knn.kneighbors(X_new, n_neighbors=1)
y_pred = dbscan.labels_[dbscan.core_sample_indices_][y_pred_idx]
y_pred[y_dist > 0.2] = -1
y_pred.ravel()

array([-1,  0,  3, -1])

### **9.1.7 다른 군집 알고리즘**

+ **병합 군집** : 클러스터 계층을 밑바닥부터 위로 쌓아 구성  
+ **BIRCH** : 대규모 데이터셋을 위해 고안되어 새로운 샘플을 클러스터에 빠르게 할당할 수 있는 정보가 담긴 트리 구조
+ **평균-이동** : 각 샘플을 중심으로 하는 원을 그려 그 내부의 샘플 평균을 구해 평균점으로 이동
+ **유사도 전파** : 투표방식을 사용하여 샘플이 자신을 대표하는 샘플에 투표하여 클러스터 형성
+ **스펙트럼 군집** : 샘플 사이의 유사도 행렬을 받아 저차원 임베딩을 통해 저차원 공간에서 다른 군집 알고리즘을 사용

## **9.2 가우시안 혼합**

**가우시안 혼합 모델 (GMM)** : 파라미터가 알려지지 않은 여러 개의 혼합된 가우시안 분포에서 샘플이 생성되었다고 가정하는 확률 모델.

In [41]:
from sklearn.mixture import GaussianMixture

gm = GaussianMixture(n_components=3, n_init=10)
gm.fit(X)

GaussianMixture(n_components=3, n_init=10)

In [42]:
gm.weights_

array([0.20859793, 0.20453936, 0.58686271])

In [43]:
gm.means_

array([[ 1.73110722, -0.06902321],
       [-0.74503074,  0.55807632],
       [ 0.4950664 ,  0.25373843]])

In [44]:
gm.covariances_

array([[[ 0.06102278,  0.06561808],
        [ 0.06561808,  0.08773031]],

       [[ 0.05459987,  0.06280737],
        [ 0.06280737,  0.09028234]],

       [[ 0.16337036, -0.09692064],
        [-0.09692064,  0.28734238]]])

In [45]:
gm.converged_

True

In [46]:
gm.n_iter_

17

In [47]:
gm.predict(X)

array([2, 2, 2, 2, 1, 2, 2, 2, 0, 2, 0, 2, 2, 0, 0, 0, 2, 0, 2, 1, 2, 2,
       1, 2, 2, 2, 2, 0, 0, 0, 1, 2, 2, 1, 1, 2, 2, 2, 0, 2, 1, 2, 1, 0,
       1, 2, 0, 2, 1, 0, 2, 0, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2,
       2, 0, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 0, 2,
       2, 1, 1, 1, 2, 2, 2, 1, 2, 2, 2, 0, 2, 1, 0, 2, 2, 2, 2, 0, 2, 0,
       2, 0, 0, 0, 2, 1, 2, 0, 2, 2, 0, 1, 2, 2, 2, 2, 2, 1, 0, 2, 2, 0,
       2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 0, 0, 0, 2, 0, 1, 2, 2, 0, 1,
       0, 1, 1, 2, 2, 0, 2, 2, 2, 2, 0, 2, 1, 2, 1, 1, 1, 1, 2, 0, 1, 1,
       2, 2, 0, 2, 1, 2, 1, 2, 0, 2, 1, 1, 0, 2, 0, 0, 2, 2, 2, 1, 1, 2,
       2, 2, 2, 2, 2, 2, 0, 2, 2, 0, 1, 0, 2, 2, 0, 0, 2, 1, 2, 2, 0, 2,
       1, 2, 0, 2, 0, 2, 2, 1, 0, 2, 1, 0, 1, 2, 2, 2, 2, 0, 2, 1, 2, 2,
       2, 2, 1, 2, 1, 2, 2, 2, 2, 0, 2, 0, 1, 1, 2, 0, 0, 2, 2, 1, 2, 2,
       1, 0, 0, 1, 2, 1, 1, 2, 0, 2, 2, 2, 2, 2, 2, 0, 2, 0, 2, 1, 2, 2,
       1, 1, 2, 0, 0, 2, 2, 1, 1, 1, 0, 1, 2, 2, 2,

In [48]:
gm.predict_proba(X)

array([[2.30146837e-081, 9.12595056e-014, 1.00000000e+000],
       [3.39933511e-004, 7.47233037e-120, 9.99660066e-001],
       [9.48577999e-007, 2.17446845e-107, 9.99999051e-001],
       ...,
       [9.84186191e-001, 1.43718998e-182, 1.58138088e-002],
       [1.66671181e-161, 9.99684465e-001, 3.15534827e-004],
       [5.19983644e-039, 4.32492679e-045, 1.00000000e+000]])

In [49]:
X_new, y_new = gm.sample(6)

In [50]:
gm.score_samples(X)

array([-1.59760943e+00, -1.83480188e+00, -1.56859578e+00, -1.90181248e+00,
       -2.43278366e-01, -1.34340986e+00, -1.58942671e+00, -1.59095129e+00,
       -3.48586901e-01, -1.79801683e+00, -1.86745681e+00, -1.66878897e+00,
       -1.44692396e+00, -4.68986083e-01, -8.67052879e-01,  2.00709206e-02,
       -2.14236134e+00, -7.08571708e-01, -1.97978983e+00, -8.67185148e-01,
       -1.76567591e+00, -1.76795375e+00, -1.20391122e-01, -1.59942948e+00,
       -1.54386695e+00, -1.76015872e+00, -1.66076963e+00, -1.22061360e+00,
       -3.22723479e-01, -5.49875652e-01, -4.98285317e-01, -1.43940247e+00,
       -1.82093613e+00, -2.02190156e+00,  8.50664496e-04, -1.96534568e+00,
       -1.57429254e+00, -1.28026138e+00, -2.83331977e-01, -1.56852030e+00,
       -1.20480578e+00, -1.66428926e+00, -1.01201107e+00, -1.19939167e+00,
       -1.43416458e+00, -2.11803830e+00, -1.70516741e+00, -1.57158084e+00,
       -3.26271946e-01, -5.77558013e-01, -1.82225216e+00, -8.95679851e-01,
       -1.96859753e+00, -

### **9.2.1 가우시안 혼합을 사용한 이상치 탐지**

임계값을 기준으로 밀도가 낮은 지역에 있는 모든 샘플을 이상치로

In [51]:
densities = gm.score_samples(X)
density_threshold = np.percentile(densities, 4)
anomalies = X[densities < density_threshold]

### **9.2.2 클러스터 개수 선택하기**

이너셔나 실루엣 점수와 같은 지표는 클러스터가 타원형이나 크기 차이가 나면 안정적이지 않으므로 가우시안 혼합에서 이런 지표를 사용할 수 없음.  
> BIC  
> AIC 

### **9.2.3 베이즈 가우시안 혼합 모델**


최적의 클러스터 개수를 수동으로 찾지 않고 불필요한 클러스터의 가중치를 0으로 만드는
> 사전 확률 : 잠재 변수 z에 대한 확률분포 인코딩  
> 사전 믿음 : 클러스터가 적고 풍부하다는 믿음

In [52]:
from sklearn.mixture import BayesianGaussianMixture
bgm = BayesianGaussianMixture(n_components=10, n_init=10)
bgm.fit(X)
np.round(bgm.weights_,2)

/usr/local/lib/python3.7/dist-packages/sklearn/mixture/_base.py:282: ConvergenceWarning: Initialization 10 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  ConvergenceWarning,


array([0.16, 0.08, 0.15, 0.13, 0.12, 0.12, 0.12, 0.  , 0.11, 0.  ])

### **9.2.4 이상치 탐지와 특이치 탐지를 위한 다른 알고리즘**

+ **PCA** : 샘플의 재구성 오차와 이상치의 재구성 오차를 비교  
+ **Fast-MCD** : 데이터셋을 정제할 때 사용하여 이상치 감지
+ **아이솔레이션 포레스트** : 고차원 데이터셋에서 이상치 감지를 위한 알고리즘  
+ **LOF** : 주어진 샘플 주위의 밀도와 이웃 주위의 밀도를 비교  
+ **one-class SVM** : 모든 샘플을 고차원 공간에 매핑한 다음 그 고차원 공간에서 선형 SVM 분류기를 사용해 두 클래스 분리.